## AGNO Secure Agent
This notebook demonstrates how to use the Agno agentic framework to make a RAG call to Gemini with a prompt injection guardrail. We'll start by installing the agno library

In [ ]:
%pip install agno chromadb pypdf

We'll now import the Agno functions including the connector to connect to Gemini. We'll also include the pipeline for our huggingface guardrail.

In [2]:
from google.colab import userdata
import chromadb
from pypdf import PdfReader
from agno.agent import Agent
from agno.models.google import Gemini
from agno.exceptions import InputCheckError
from agno.guardrails import BaseGuardrail
from agno.run.agent import RunInput
from transformers import pipeline

We'll create our vector database and preload the pdf document.  make sure the document has been uploaded to Colab before running this cell.

In [3]:
client     = chromadb.Client()
collection = client.create_collection(name="handbook")
reader = PdfReader("GAIA_Quality_Manual.pdf")

texts=[]
metas=[]
ideas=[]
for page in reader.pages:
    texts.append(page.extract_text())
    metas.append({"source":"page"+str(len(texts))})
    ideas.append("idea"+str(len(texts)))
    collection.add(
        documents = texts,
        metadatas = metas,
        ids       = ideas)

/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:01<00:00, 73.8MiB/s]


We'll create a pipeline handle for the ProtectAI Deberta prompt injection guardrail.

In [ ]:
pipe1 = pipeline("text-classification", model="protectai/deberta-v3-base-prompt-injection-v2")

We'll define the pre AI model guardrail prompt injection function as a class. We'll use Agno's custom tool approach, and we need to define both sync and async calling methods.

In [5]:
class PreGuardrail(BaseGuardrail):
    def check(self, run_input: RunInput) -> None:
        if isinstance(run_input.input_content, str):
              if pipe1(run_input.input_content)[0]['label']== "INJECTION":
                 raise InputCheckError("User appears to be attempting a prompt injection.")
    async def async_check(self, run_input: RunInput) -> None:
        self.check(run_input)

We'll set up our system instructions for Gemini.

In [6]:
system_instructions = """
   You are a helpful business architect who can identify business requirements.  You analyze the provided Context
   and select requirements which exist in the Context in order to answer the user Question This is the only place
   the requirements exist so do not use your own knowledge to answer the question, use only the Context.
   Attributes are single word adjectives describing a requirement. Identify any key performance attributes
   related to the requirements and the list of single word attributes that relate to them.
   You always return a json string in the following form:
   { 'requirement':'requirement 1','attribute_list':['attribute 1','attribute 2',..],..}
   If you cannot find the answer in the Context then reply that you do not know.
   """

We'll now build the agent using Agno and specify that the LLM we'll use is gemini-2.5-flash, and we'll use our pre-model guardrail.

In [7]:
preguard = PreGuardrail()
agent = Agent(
    model=Gemini(id="gemini-2.5-flash",api_key=userdata.get("GEMINI_API_KEY")),
    instructions=[system_instructions],
    description="You are a helpful research assistant",
    pre_hooks=[preguard],
    markdown=True
)

We'll pose a question that we'll have the agent answer.

In [8]:
query = "What skills should Laboratory employees have?"

We now need to get our context and construct our rich context-enhanced prompt.

In [9]:
results = collection.query(query_texts=[query],n_results=5)
docs = results["documents"]
context = ""
for doc in docs:
    context = context.join(doc[0])

prompt  = f"""
   Answer the question based on the Context.
   Context:
   {context}
   Question:
   {query}
   """

We can now make a request and stream the result out for display.

In [ ]:
agent.print_response(prompt, stream=True)